In [ ]:
import os
import bs4

from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings 
from langchain_community.embeddings import HuggingFaceEmbeddings 
from sentence_transformers import SentenceTransformer

URL_TO_SCRAPE = "https://lilianweng.github.io/posts/2023-06-23-agent/"
MODELSCOPE_ACCESS_TOKEN = "ms-6634b7ff-46bc-4400-b9f0-d475b0890dab" 
MODELSCOPE_BASE_URL = "https://api-inference.modelscope.cn/v1/"
EMBEDDING_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2" 
CHROMA_DB_PATH = "./chroma_db_multilingual_minilm" 

print("--- 1. 正在抓取网页内容... ---")

bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=(URL_TO_SCRAPE,),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

print("--- 2. 正在进行文本分块 (Chunking) 和向量化 (Embedding)... ---")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=200,
    add_start_index=True
)
splits = text_splitter.split_documents(docs)

embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    model_kwargs={'device': 'cuda'},
    encode_kwargs={'normalize_embeddings': True}
)

vectorstore = Chroma.from_documents(
    documents=splits, 
    embedding=embeddings, 
    persist_directory=CHROMA_DB_PATH
)

print(f"成功创建 {len(splits)} 个文本块并存入向量数据库。")
print("--- 数据处理完成。 ---")

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

USER_AGENT environment variable not set, consider setting it to identify your requests.
d:\env\py\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


--- 1. 正在抓取网页内容... ---
--- 2. 正在进行文本分块 (Chunking) 和向量化 (Embedding)... ---


C:\Users\Admin\AppData\Local\Temp\ipykernel_16032\1718351926.py:35: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


ValueError: Expected Embeddings to be non-empty list or numpy array, got [] in upsert.

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.retrieval import create_retrieval_chain

llm = ChatOpenAI(
    model="Qwen/Qwen2.5-Coder-32B-Instruct", 
    base_url=MODELSCOPE_BASE_URL, 
    api_key=MODELSCOPE_ACCESS_TOKEN,
    temperature=0
) 

system_prompt = (
    "你是一位专业的 AI 助理。请根据提供的上下文 (Context) 来回答问题。如果上下文中没有信息，"
    "请说明你无法找到答案。请用中文简洁明了地回答问题，并引用上下文中的关键信息。"
    "\n\nContext: {context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

print("--- RAG 问答链构建完成。 ---")

question_a = "什么是 Task Decomposition (任务分解)？它在 AI Agent 中有什么作用？"
print(f"问题 A: {question_a}")

response_a = rag_chain.invoke({"input": question_a})
print("\n--- 回答 A ---")
print(response_a['answer'])
print("----------------\n")


question_b = "这篇博客的作者是谁？"
print(f"问题 B: {question_b}")

response_b = rag_chain.invoke({"input": question_b})
print("\n--- 回答 B ---")
print(response_b['answer'])
print("----------------\n")


# 删除本地 ChromaDB 文件夹
# import shutil
# shutil.rmtree("./chroma_db")
# print("本地数据库已清理。")

--- RAG 问答链构建完成。 ---
问题 A: 什么是 Task Decomposition (任务分解)？它在 AI Agent 中有什么作用？

--- 回答 A ---
Task Decomposition（任务分解）是一种将复杂任务分解为更小、更简单步骤的方法。这种方法通过指导模型“逐步思考”来利用更多的计算资源，从而提高模型在复杂任务上的表现。任务分解将大型任务转换为多个可管理的任务，并提供了对模型思维过程的洞察。

在 AI Agent 中，任务分解的作用是帮助代理了解需要执行的各个步骤，并提前进行规划。这使得复杂的任务可以被系统地处理，提高了任务执行的效率和准确性。通过将任务分解成更小的部分，AI Agent 可以更容易地管理和优化每个子任务的执行过程。
----------------

问题 B: 这篇博客的作者是谁？

--- 回答 B ---
这篇博客的作者是 Weng, Lilian。相关引用信息如下：
```
@article{weng2023agent,
  title   = "LLM-powered Autonomous Agents",
  author  = "Weng, Lilian",
  journal = "lilianweng.github.io",
  year    = "2023",
  month   = "Jun",
  url     = "https://lilianweng.github.io/posts/2023-06-23-agent/"
}
```
----------------

